In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:

# Load the first dataframe and drop specified columns
df_all = pd.read_csv('all_timeslot_cleaned.csv')
df_all = df_all.drop(['gps_X', 'gps_Y', 'Filename'], axis=1)  # Combined drop for efficiency

# Load the second dataframe
df_all_d = pd.read_csv('all_timeslot_derivative_data_cleaned.csv')

# Remove rows with any NaN values from both dataframes
df_all = df_all.dropna(axis=0, how='any')  # axis=0 drops rows, how='any' drops if any NaN exists
df_all_d = df_all_d.dropna(axis=0, how='any')

# Optional: Verify the result (check if any NaNs remain)
print(f"df_all remaining NaNs: {df_all.isna().sum().sum()}")
print(f"df_all_d remaining NaNs: {df_all_d.isna().sum().sum()}")


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
df_all

In [ ]:
df_all_d

In [ ]:
## X is the actual data

In [ ]:

X_all = df_all.iloc[:, 7:]

In [ ]:


scaler_X_all = StandardScaler()
X_all_scaled = scaler_X_all.fit_transform(X_all)
X_all_scaled = pd.DataFrame(X_all_scaled, columns=X_all.columns)

In [ ]:

Y_all = df_all.iloc[:, :7]

In [ ]:


scaler_Y_all = StandardScaler()
Y_all_scaled = scaler_Y_all.fit_transform(Y_all)
Y_all_scaled = pd.DataFrame(Y_all_scaled, columns=Y_all.columns)


In [ ]:
## X_d is the 

In [ ]:

X_all_d = df_all_d.iloc[:, 7:]

In [ ]:

scaler_X_all_d = StandardScaler()
X_all_d_scaled = scaler_X_all_d.fit_transform(X_all_d)
X_all_d_scaled = pd.DataFrame(X_all_d_scaled, columns=X_all_d.columns)

In [ ]:

Y_all_d = df_all_d.iloc[:, :7]

In [ ]:

scaler_Y_all_d = StandardScaler()
Y_all_d_scaled = scaler_Y_all_d.fit_transform(Y_all_d)
Y_all_d_scaled = pd.DataFrame(Y_all_d_scaled, columns=Y_all_d.columns)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# # Set the size of the figure
plt.figure(figsize=(10, 5))

# Create the heatmap with enhancements
sns.heatmap(X_all.corr(), annot=True, fmt=".2f", cmap='coolwarm', 
            linewidths=0.5,)

# Add a title with enhanced formatting
plt.title('Correlation Heatmap', fontweight='bold')

# Show the plot
plt.show()

## 用linear regression 测试关系

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (
    r2_score, mean_absolute_error, mean_squared_error,
    explained_variance_score, median_absolute_error
)

def evaluate_linear_regression(X, y, test_size=0.1, cv_folds=5, name='VOC'):
    
    # Split into train/test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, shuffle=True)
    print(f'Training size: {len(X_train)}, Testing size: {len(X_test)}')

    # Instantiate and fit the model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Calculate metrics
    r2_model = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)  # Calculate RMSE
    explained_var = explained_variance_score(y_test, y_pred)
    median_ae = median_absolute_error(y_test, y_pred)

    # Baseline model: Mean prediction
    y_baseline = np.mean(y_train)  # Simple baseline prediction
    y_baseline_pred = np.full(y_test.shape, y_baseline)

    # Calculate R² for both models
    r2_baseline = r2_score(y_test, y_baseline_pred)

    # Calculate Adjusted R²
    n = X_test.shape[0]  # Number of data points
    p = X_test.shape[1]  # Number of predictors
    adjusted_r2 = 1 - (1 - r2_model) * (n - 1) / (n - p - 1)

    # Cross-validation for R²
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv_folds, scoring='r2', verbose=2)
    mean_cv_r2 = np.mean(cv_scores)

    # Print results
    print(f"R² for Baseline Model: {r2_baseline:.4f}")
    print(f"R² for Linear Regression Model: {r2_model:.4f}")
    print(f"Adjusted R² for Linear Regression Model: {adjusted_r2:.4f}")
    print(f"Mean Cross-Validated R²: {mean_cv_r2:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"Explained Variance Score: {explained_var:.4f}")
    print(f"Median Absolute Error: {median_ae:.4f}")

    # # Visualization
    sns.set(style="whitegrid")
    fig, axs = plt.subplots(2, 2, figsize=(18, 14))

    # 1. Scatter plot: True vs Predicted
    axs[0, 0].scatter(y_test, y_pred, alpha=0.6, color="#0072B2", edgecolor='k', s=60)
    axs[0, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=3)
    axs[0, 0].set_xlabel("True Values", fontsize=14)
    axs[0, 0].set_ylabel("Predicted Values", fontsize=14)
    axs[0, 0].set_title(f"True vs Predicted Values ({name})", fontsize=16, fontweight='bold')
    axs[0, 0].grid(True, which='both', linestyle='--', linewidth=0.5)
    axs[0, 0].tick_params(axis='both', labelsize=12)
    min_val = min(y_test.min(), y_pred.min())
    max_val = max(y_test.max(), y_pred.max())
    axs[0, 0].set_xlim(min_val, max_val)
    axs[0, 0].set_ylim(min_val, max_val)
    axs[0, 0].set_aspect('equal', adjustable='box')

    # 2. Prediction error plot (regression line)
    sns.regplot(
        x=y_test, y=y_pred, ax=axs[0, 1],
        scatter_kws={"alpha": 0.6, "color": "#D55E00", "edgecolor": "k", "s": 60},
        line_kws={"color": "navy", "lw": 3}
    )
    axs[0, 1].set_xlabel("True Values", fontsize=14)
    axs[0, 1].set_ylabel("Predicted Values", fontsize=14)
    axs[0, 1].set_title(f"Prediction Error Plot ({name})", fontsize=16, fontweight='bold')
    axs[0, 1].grid(True, linestyle='--', linewidth=0.5)
    axs[0, 1].tick_params(axis='both', labelsize=12)

    # 3. Feature importances (for linear regression, coefficients can be viewed)
    coefficients = model.coef_
    features = X.columns
    indices = np.argsort(coefficients)[::-1]
    # Create a color palette
    colors = sns.color_palette("coolwarm", len(coefficients))
    bars = axs[1, 0].barh(range(len(coefficients)), coefficients[indices][::-1], color=colors[::-1], edgecolor='black')
    axs[1, 0].set_yticks(range(len(coefficients)))
    axs[1, 0].set_yticklabels(features[indices][::-1], fontsize=12)
    axs[1, 0].invert_yaxis()  # Highest importance on top
    axs[1, 0].set_xlabel("Coefficient Value", fontsize=14)
    axs[1, 0].set_title(f"Linear Regression Coefficients ({name})", fontsize=16, fontweight='bold')
    axs[1, 0].grid(axis='x', linestyle='--', linewidth=0.5)

    # 4. Distribution of Actual vs Predicted VOC (KDE plot)
    sns.kdeplot(y_test, label=f'Actual {name}', fill=True, ax=axs[1, 1], color="#1B9E77")
    sns.kdeplot(y_pred, label=f'Predicted {name}', fill=True, ax=axs[1, 1], color="#D95F02")
    axs[1, 1].set_title(f'Distribution of Actual vs Predicted ({name})', fontsize=16, fontweight='bold')
    axs[1, 1].set_xlabel(f'{name}', fontsize=14)
    axs[1, 1].legend(fontsize=12)
    axs[1, 1].tick_params(axis='both', labelsize=12)
    axs[1, 1].grid(True, linestyle='--', linewidth=0.5)

    # plt.tight_layout()
    plt.show()
    
    return {
        'r2': r2_model,
        'adjusted_r2': adjusted_r2,
        'r2_baseline': r2_baseline,
        'mae': mae,
        'mse': mse,
        'rmse': rmse,
        'explained_var': explained_var,
        'median_ae': median_ae,
        'cv_r2': mean_cv_r2
    }

In [ ]:
from typing import List
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

def results_summary(
    air_type: str, 
    X_in: List[pd.DataFrame], 
    Y_in: List[pd.DataFrame], 
    X_in_names: List[str], 
    Y_in_names: List[str]
) -> None:
    """
    Generate a results summary matrix and heatmap for given X and Y inputs.
    
    Args:
        air_type (str): The target air quality type (e.g., 'VOC').
        X_in (list): List of feature input DataFrames.
        Y_in (list): List of target input DataFrames.
        X_in_names (list): Names corresponding to X_in.
        Y_in_names (list): Names corresponding to Y_in.
    """
    # Initialize an empty DataFrame for results
    results_matrix = pd.DataFrame(index=X_in_names, columns=Y_in_names)
    
    # Fill the matrix with evaluation results
    for i, x in enumerate(tqdm(X_in, desc="X Loop")):
        for j, y in enumerate(tqdm(Y_in, desc="Y Loop")):
            try:
                print(f"\n → X: {X_in_names[i]}, Y: {Y_in_names[j]}")
                metrics = evaluate_linear_regression(x, y[air_type], name=air_type)
                results_matrix.iloc[i, j] = round(metrics['r2'], 4) 
            except ValueError as e:
                # Handle inconsistent sample size or other ValueErrors
                print(f"Skipping pair (X: {X_in_names[i]}, Y: {Y_in_names[j]}) due to error: {e}")
                results_matrix.iloc[i, j] = None  # Mark as empty or 'None'

    # Display the results matrix
    print("\nResults Summary Matrix:")
    print(results_matrix)

    # Plot a heatmap, skipping NaN values
    plt.figure(figsize=(4, 3))
    sns.heatmap(
        results_matrix.astype(float), 
        annot=True, 
        cmap='coolwarm', 
        xticklabels=Y_in_names, 
        yticklabels=X_in_names, 
        cbar_kws={'label': 'R² Value'},
        mask=results_matrix.isnull()  # Mask NaN values
    )
    plt.title(f'R² Values Heatmap for {air_type}')
    plt.xlabel('Y Inputs')
    plt.ylabel('X Inputs')
    plt.show()

# Define placeholders for potentially missing variables
X_3 = X_3_scaled = X_3_d = X_3_d_scaled = None
Y_3 = Y_3_scaled = Y_3_d = Y_3_d_scaled = None

def choose_timeslot(
    timeslot: str = '1', 
    air_type: str = 'VOC'
) -> None:
    """
    Choose the appropriate timeslot and generate results summary.

    Args:
        timeslot (str): Timeslot to use ('1', '2', '3', or 'all').
        air_type (str): The target air quality type (e.g., 'VOC').
    """
    # Define the timeslot mapping
    timeslot_mapping = {
        # '1': {
        #     'X_input': [X_1, X_1_scaled, X_1_d, X_1_d_scaled],
        #     'Y_input': [Y_1, Y_1_scaled, Y_1_d, Y_1_d_scaled],
        #     'X_names': ['X_1', 'X_1_scaled', 'X_1_d', 'X_1_d_scaled'],
        #     'Y_names': ['Y_1', 'Y_1_scaled', 'Y_1_d', 'Y_1_d_scaled']
        # },
        # '2': {
        #     'X_input': [X_2, X_2_scaled, X_2_d, X_2_d_scaled],
        #     'Y_input': [Y_2, Y_2_scaled, Y_2_d, Y_2_d_scaled],
        #     'X_names': ['X_2', 'X_2_scaled', 'X_2_d', 'X_2_d_scaled'],
        #     'Y_names': ['Y_2', 'Y_2_scaled', 'Y_2_d', 'Y_2_d_scaled']
        # },
        # '3': {
        #     'X_input': [X_3, X_3_scaled, X_3_d, X_3_d_scaled],
        #     'Y_input': [Y_3, Y_3_scaled, Y_3_d, Y_3_d_scaled],
        #     'X_names': ['X_3', 'X_3_scaled', 'X_3_d', 'X_3_d_scaled'],
        #     'Y_names': ['Y_3', 'Y_3_scaled', 'Y_3_d', 'Y_3_d_scaled']
        # },
        'all': {
            'X_input': [X_all, X_all_scaled, X_all_d, X_all_d_scaled],
            'Y_input': [Y_all, Y_all_scaled, Y_all_d, Y_all_d_scaled],
            'X_names': ['X_all', 'X_all_scaled', 'X_all_d', 'X_all_d_scaled'],
            'Y_names': ['Y_all', 'Y_all_scaled', 'Y_all_d', 'Y_all_d_scaled']
        }
    }
    
    # Ensure the timeslot is valid
    if timeslot not in timeslot_mapping:
        raise ValueError(f"Invalid timeslot '{timeslot}'. Choose from '1', '2', '3', or 'all'.")

    # Get inputs and names for the selected timeslot
    selected_timeslot = timeslot_mapping[timeslot]
    X_input = selected_timeslot['X_input']
    Y_input = selected_timeslot['Y_input']
    X_names = selected_timeslot['X_names']
    Y_names = selected_timeslot['Y_names']

    # Generate the results summary
    results_summary(air_type=air_type, X_in=X_input, Y_in=Y_input, X_in_names=X_names, Y_in_names=Y_names)

## ALL Time slot

In [ ]:
choose_timeslot('all','VOC')

In [ ]:
choose_timeslot('all','CO')

In [ ]:
choose_timeslot('all','NO2')

In [ ]:
choose_timeslot('all','SO2')

In [ ]:
choose_timeslot('all','PM25')

In [ ]:
choose_timeslot('all','PM10')

In [ ]:
choose_timeslot('all','O3')